In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [33]:
version = 'v2'

# R  W  1111111111
# R  P  111111111
# G  W  111111111
# G  P  111111111
# H  W  111111111
# H  P  111111111

category = 'R'
# category = 'G'
# category = 'H'

bet_type = 'W'
# bet_type = 'P'

In [34]:
engine = create_engine('sqlite:///../../data/race.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

32545 races loaded!


In [35]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

10456 R races!


In [36]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head(3)

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
0,1,BALLINA,NSW,C,R,2017-10-01,1,COATES HIRE MAIDEN PLATE,2017-10-01 01:40:00,Paying,1600,"[[4], [9], [7], [6]]",11.0,"[{""runnerName"": ""WHO AM I"", ""runnerNumber"": 12...",35.2,27.4,170.9,401.1,"[[4], [9], [7], [6]]","[{'runnerName': 'WHO AM I', 'runnerNumber': 12..."
1,3,BALLINA,NSW,C,R,2017-10-01,3,CNW ELECTRICAL WHOLESALE CG&E BM55,2017-10-01 02:55:00,Paying,1000,"[[3], [4], [5], [1]]",7.0,"[{""runnerName"": ""MCLENNAN"", ""runnerNumber"": 1,...",14.1,23.1,100.3,710.5,"[[3], [4], [5], [1]]","[{'runnerName': 'MCLENNAN', 'runnerNumber': 1,..."
2,4,BALLINA,NSW,C,R,2017-10-01,4,LEND LEASE MAIDEN HANDICAP,2017-10-01 03:35:00,Paying,1300,"[[14], [5], [1], [13]]",12.0,"[{""runnerName"": ""WOODDITTON"", ""runnerNumber"": ...",8.5,22.6,49.6,1158.9,"[[14], [5], [1], [13]]","[{'runnerName': 'WOODDITTON', 'runnerNumber': ..."


In [37]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
    data_all = data_all.append(race['runners'])
data_all.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,has_odds,num_runners,...,place_rank,place_scaled,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
10,NaN,NaN,NaN,NaN,6,-1.0,0,"{'returnWin': 15, 'returnWinOpen': 12, 'return...",True,0.083333,...,0.692308,0.064811,JOSEPH TALAMO,SPECIAL STORY,6,RONALD W ELLIS,16.3,0.061350,0.615385,0.051913
11,NaN,NaN,NaN,NaN,7,-1.0,0,"{'returnWin': 9.5, 'returnWinOpen': 8, 'return...",True,0.083333,...,0.538462,0.039252,COREY NAKATANI,SUPER ECHELON,7,CRAIG A LEWIS,18.3,0.054645,0.461538,0.046239
12,NaN,NaN,NaN,NaN,1,-1.0,0,"{'returnWin': 71, 'returnWinOpen': 35, 'return...",True,0.083333,...,0.538462,0.039252,DRAYDEN VAN DYKE,RED CARPET CAT,1,VANN BELVOIR,44.6,0.022422,0.307692,0.018973


In [38]:
# drop scratched
data = data_all.dropna(subset=['has_odds'])
data.tail(3)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,has_odds,num_runners,...,place_rank,place_scaled,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
10,NaN,NaN,NaN,NaN,6,-1.0,0,"{'returnWin': 15, 'returnWinOpen': 12, 'return...",True,0.083333,...,0.692308,0.064811,JOSEPH TALAMO,SPECIAL STORY,6,RONALD W ELLIS,16.3,0.061350,0.615385,0.051913
11,NaN,NaN,NaN,NaN,7,-1.0,0,"{'returnWin': 9.5, 'returnWinOpen': 8, 'return...",True,0.083333,...,0.538462,0.039252,COREY NAKATANI,SUPER ECHELON,7,CRAIG A LEWIS,18.3,0.054645,0.461538,0.046239
12,NaN,NaN,NaN,NaN,1,-1.0,0,"{'returnWin': 71, 'returnWinOpen': 35, 'return...",True,0.083333,...,0.538462,0.039252,DRAYDEN VAN DYKE,RED CARPET CAT,1,VANN BELVOIR,44.6,0.022422,0.307692,0.018973


In [39]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    116373.000000
mean          0.089969
std           0.286139
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: finishingPosition, dtype: float64

In [40]:
xn = data['num_runners']
xwp = data['win_perc']
xws = data['win_scaled']
xwr = data['win_rank']
xpp = data['place_perc']
xps = data['place_scaled']
xpr = data['place_rank']
X = pd.concat([xn, xwp, xws, xwr, xpp, xps, xpr], axis=1)
X.describe()

,num_runners,win_perc,win_scaled,win_rank,place_perc,place_scaled,place_rank
count,116373.000000,116373.000000,116373.000000,116373.000000,116373.000000,116373.000000,116373.000000
mean,0.102081,0.106311,0.089849,0.528438,0.295034,0.089849,0.532775
std,0.031916,0.117095,0.098739,0.314093,0.229317,0.074456,0.315674
min,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.083333,0.025510,0.021530,0.285714,0.117647,0.034303,0.285714
50%,0.090909,0.068027,0.057524,0.545455,0.250000,0.074751,0.555556
75%,0.111111,0.147059,0.124601,0.800000,0.434783,0.129323,0.800000
max,0.333333,0.961538,0.894509,1.000000,1.000000,0.946292,1.000000


In [41]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 500
print('epochs = {}'.format(epochs))

layer_1 = 50
layer_2 = 50

tag = '{}{}x{}{}'.format(category, layer_1, layer_2, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/each_way/{}/models/{}.h5'.format(version, tag)

try:
    model = load_model(file_name)
    print('model loaded')
except OSError:
    model = Sequential()
    model.add(Dense(layer_1, activation='relu', input_dim=n))
    model.add(Dense(layer_2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    print('model created')

input dimension = 7
epochs = 500
tag = R50x50W
model loaded


In [42]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/each_way/{}/summary/{}'.format(version, tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file
model.save(file_name)
print('model saved')


model saved
